# 라이브러리

In [19]:
# 기본 설정
import random
import os
import warnings ; warnings.filterwarnings('ignore')

# 데이터 처리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 100)

# 데이터 전처리
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.base import clone, BaseEstimator, TransformerMixin

# kfold
from sklearn.model_selection import StratifiedKFold , KFold

# oversampling
from imblearn.over_sampling import SMOTE, SMOTENC, ADASYN, SVMSMOTE
# 평가지표
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, f1_score, log_loss, make_scorer
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve, average_precision_score

# modeling
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from xgboost import XGBRegressor, XGBClassifier
import optuna
from imblearn.over_sampling import SMOTE, SMOTENC, ADASYN, SVMSMOTE

# seed 고정

In [20]:
# Reproducibility setup
RANDOM_SEED = 42
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# 데이터 불러오기

In [21]:
train = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/train.csv").drop("~", axis = 1) #경로지정
test = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/test.csv").drop("No", axis = 1)
sample_sub = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/submission.csv")

# 파라미터 설정

In [22]:
TARGET = "Death"

# 전처리


## EDA

## feature engineering


In [23]:
def preprocessing(data):
    # Lung cancer TNM stage
    #T
    data.loc[(data['M1a'] == 1 ) | (data['M1b'] == 1) | (data['M1c'] == 1), 'M0' ] = 0 
    data.loc[(data['M1a'] == 0) & (data['M1b'] == 0) & (data['M1c'] == 0),'M0'] = 1

    #N
    data.loc[(data['N1'] == 1 ) | (data['N2'] == 1) | (data['N3'] == 1), 'N0'] = 0
    data.loc[(data['N1'] == 0) & (data['N2'] == 0) & (data['N3'] == 0), 'N0'] = 1
    #M
    
    #stage
    data['TNM_stage'] = ''

    stage_map = {

        ('T1', 'N0'): 'IA', ('T1', 'N1'): 'IIA', ('T1', 'N2'): 'IIB', ('T1', 'N3'): 'IIIA', 
        ('T1a', 'N0'): 'IA', ('T1a', 'N1'): 'IIA', ('T1a', 'N2'): 'IIB', ('T1a', 'N3'): 'IIIA',
        ('T1b', 'N0'): 'IA', ('T1b', 'N1'): 'IIA', ('T1b', 'N2'): 'IIB', ('T1b', 'N3'): 'IIIA', 
        ('T1c', 'N0'): 'IA', ('T1c', 'N1'): 'IIA', ('T1c', 'N2'): 'IIB', ('T1c', 'N3'): 'IIIA', 
        ('T2', 'N0'): 'IB', ('T2', 'N1'): 'IIB', ('T2', 'N2'): 'IIIA', ('T2', 'N3'): 'IIIB',
        ('T2a', 'N0'): 'IB', ('T2a', 'N1'): 'IIB', ('T2a', 'N2'): 'IIIA', ('T2a', 'N3'): 'IIIB', 
        ('T2b', 'N0'): 'IIA', ('T2b', 'N1'): 'IIB', ('T2b', 'N2'): 'IIIA', ('T2b', 'N3'): 'IIIB', 
        ('T3', 'N0'): 'IIB', ('T3', 'N1'): 'IIIA', ('T3', 'N2'): 'IIIA', ('T3', 'N3'): 'IIIB',
        ('T4', 'N0'): 'IIIA', ('T4', 'N1'): 'IIIA', ('T4', 'N2'): 'IIIB', ('T4', 'N3'): 'IIIC'
    }

    for t in ['T1', 'T1a', 'T1b', 'T1c', 'T2', 'T2a', 'T2b', 'T3', 'T4']:
        for n in ['N0', 'N1', 'N2', 'N3']:
            mask = (data[t] == 1) & (data[n] == 1)
            target = stage_map.get((t, n), 'Invalid TNM')
            data.loc[mask, 'TNM_stage'] = target
            if target == 'Invalid TNM':
                print((t, n))

    data.loc[data['M1a'] == 1, 'TNM_stage'] = 'IVA'
    data.loc[data['M1b'] == 1, 'TNM_stage'] = 'IVA'
    data.loc[data['M1c'] == 1, 'TNM_stage'] = 'IVB'

    stage_encode = {'' : 0, 'IA' : 1, "IB" : 2, "IIA" : 3, "IIB" : 4 , "IIIA" : 5, "IIIB" : 6, "IIIC": 7, "IVA": 8, "IVB":9}
    stage_encode2 = {0 : 0, 1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:4, 9:4}
    data['TNM_stage'] = data['TNM_stage'].map(stage_encode)
    data['TNM_bigstage'] = data['TNM_stage'].map(stage_encode2)
    
    # Surgery
    data['Therapy_count'] = data['Operation'] + data['Chemotherapy'] + data['Radiation Therapy']

    # Other
    data['BMI'] = data['Weight'] / (data['Height'] / 100) ** 2

    # EGFR mutation Detection encoding 99 -> 0
    data['EGFR mutation Detection'] = data['EGFR mutation Detection'].replace(99, 2)

    # Type of Drink encoding 99 -> 0
    data['Type of Drink'] = data['Type of Drink'].replace(99, 0)

    # Mu
    return data

In [24]:
train = preprocessing(train)
test = preprocessing(test)

# MODEl

In [25]:
def weighted_accuracy(y_true, y_pred):

    # 양성 클래스(1)을 정확하게 예측한 경우의 가중치
    weight_1_correct = 1
    # 음성 클래스(0)을 정확하게 예측한 경우의 가중치
    weight_0_correct = 0
    # 양성 클래스(1)을 음성 클래스(0)으로 잘못 예측한 경우의 가중치
    weight_1_incorrect = 2
    # 음성 클래스(0)을 양성 클래스(1)으로 잘못 예측한 경우의 가중치
    weight_0_incorrect = 2
    
    # 실제 양성 클래스(1)이고 모델이 양성 클래스로 예측한 경우
    correct_predictions_1 = (y_true == 1) & (y_pred == 1)
    # 실제 음성 클래스(0)이고 모델이 음성 클래스로 예측한 경우
    correct_predictions_0 = (y_true == 0) & (y_pred == 0)
    # 실제 양성 클래스(1)이고 모델이 음성 클래스로 잘못 예측한 경우
    incorrect_predictions_0 = (y_true == 1) & (y_pred == 0)
    # 실제 음성 클래스(0)이고 모델이 양성 클래스로 잘못 예측한 경우
    incorrect_predictions_1 = (y_true == 0) & (y_pred == 1)
    
    # 각 샘플에 대한 가중치 설정
    weight = np.ones_like(y_true)
    weight[correct_predictions_1] = weight_1_correct
    weight[correct_predictions_0] = weight_0_correct
    weight[incorrect_predictions_0] = weight_0_incorrect
    weight[incorrect_predictions_1] = weight_1_incorrect
    
    # 가중치를 적용한 정확도 계산
    weighted_acc = accuracy_score(y_true, y_pred, sample_weight=weight)
    return weighted_acc


In [76]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)

def cross_val_model(train, estimators, cv = skf, verbose = True, oversampling = False):
    X = train.copy()
    y = X.pop(TARGET) # 타겟 컬럼

    test_predictions = np.zeros((len(test)))

    val_scores = []
    val_acc = []
    for fold, (train_ind, valid_ind) in enumerate(skf.split(X,y)): # 그룹 지정
        model = clone(estimators)

        X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
        X_valid, y_valid = X.iloc[valid_ind], y.iloc[valid_ind]

        if oversampling:
            smotenc = ADASYN(random_state=42)
            X_train, y_train = smotenc.fit_resample(X_train, y_train)

        model.fit(X_train, y_train)

        if verbose:
            print("-" * 100)
            print(f"Fold: {fold}")

            y_preds_t = model.predict(X_train)

            y_preds_v = model.predict(X_valid)

            print(f"Train Accuracy Score:-{accuracy_score(y_train, y_preds_t)}")
            print(f"Valid Accuracy Score:-{accuracy_score(y_valid, y_preds_v)}")
            print("----------Confusion Matrix--------")
            print(confusion_matrix(y_valid, y_preds_v))
            print("-" * 100)

        test_predictions += model.predict_proba(test).T[1,:] / n_splits
        y_preds = model.predict(X_valid)
        
        val_scores.append(f1_score(y_valid, y_preds))
        val_acc.append(accuracy_score(y_valid, y_preds))

    if verbose:
        print(f"Accuracy Score: {np.array(val_acc).mean()}")
        print(f"Custom matrix: {np.array(val_scores).mean()}")

    return val_scores, test_predictions

## optuna 설정

In [77]:
def objective(trial):
    #params for XGB
    params = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if params["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        params["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        params["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if params["booster"] == "dart":
        params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    """
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 10000),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0),
        "gamma": trial.suggest_float("gamma", 1e-3, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 1e2, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 1e2, log=True),
        "random_state": RANDOM_SEED,
        "scale_pos_weight": trial.suggest_float("scale_pos_weight",2,4),
        "n_jobs": -1,
        "verbosity": 0,
    }
    """
    #params for Catboost
    """
    params = {
        "iterations": 1000,
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 1e-1, log=True),
        "border_count": trial.suggest_int("border_count", 1, 255),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 1e2, log=True),
        "leaf_estimation_iterations": trial.suggest_int("leaf_estimation_iterations", 1, 10),
        "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 0, 25),
        "random_state": RANDOM_SEED,
        "verbose": 0,
    }
    """

    model = make_pipeline(
        XGBClassifier(**params)
        )

    val_scores, _ = cross_val_model(train, model, verbose=True, oversampling=True)
    return np.mean(val_scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2024-05-13 14:10:38,884] A new study created in memory with name: no-name-a190dc02-6e0c-4820-8e1e-0f8bed9b8454


----------------------------------------------------------------------------------------------------
Fold: 0
Train Accuracy Score:-0.8112201530020864
Valid Accuracy Score:-0.7415
----------Confusion Matrix--------
[[1456  124]
 [ 393   27]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 1
Train Accuracy Score:-0.8143499147683249
Valid Accuracy Score:-0.733
----------Confusion Matrix--------
[[1431  149]
 [ 385   35]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 2
Train Accuracy Score:-0.8125482923813939
Valid Accuracy Score:-0.7475
----------Confusion Matrix--------
[[1458  121]
 [ 384   37]]
----------------------------------------------------------------------------

[I 2024-05-13 14:10:39,844] Trial 0 finished with value: 0.10461388505984166 and parameters: {'booster': 'gbtree', 'lambda': 0.00010469351510104376, 'alpha': 0.2403681062314383, 'subsample': 0.7386136091551047, 'colsample_bytree': 0.25228460977931244, 'max_depth': 3, 'min_child_weight': 5, 'eta': 0.01769302354221347, 'gamma': 1.5964323374492072e-05, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.10461388505984166.


----------------------------------------------------------------------------------------------------
Fold: 4
Train Accuracy Score:-0.8072356810524627
Valid Accuracy Score:-0.7605
----------Confusion Matrix--------
[[1494   85]
 [ 394   27]]
----------------------------------------------------------------------------------------------------
Accuracy Score: 0.746
Custom matrix: 0.10461388505984166
----------------------------------------------------------------------------------------------------
Fold: 0
Train Accuracy Score:-0.5179661540839193
Valid Accuracy Score:-0.286
----------Confusion Matrix--------
[[ 194 1386]
 [  42  378]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 1
Train Accuracy Score:-0.5212304354563769
Valid Accuracy Score:-0.363
----------Confusion Matrix--------
[[ 444 1136]
 [ 138  282]]
--------------------

[I 2024-05-13 14:10:40,277] Trial 1 finished with value: 0.288040950897207 and parameters: {'booster': 'gblinear', 'lambda': 0.004843551467069787, 'alpha': 0.09674680464792931, 'subsample': 0.3484240945361654, 'colsample_bytree': 0.8618688190767588}. Best is trial 1 with value: 0.288040950897207.


...

----------------------------------------------------------------------------------------------------
Fold: 2
Train Accuracy Score:-0.5678411373821666
Valid Accuracy Score:-0.423
----------Confusion Matrix--------
[[ 572 1007]
 [ 147  274]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 3
Train Accuracy Score:-0.5222935044105854
Valid Accuracy Score:-0.676
----------Confusion Matrix--------
[[1238  341]
 [ 307  114]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 4
Train Accuracy Score:-0.5388256056473608
Valid Accuracy Score:-0.6425
----------Confusion Matrix--------
[[1158  421]
 [ 294  127]]
-----------------------------------------------------------------------------

[I 2024-05-13 14:12:38,815] Trial 98 finished with value: 0.25601790908036326 and parameters: {'booster': 'gblinear', 'lambda': 2.211125714653114e-07, 'alpha': 0.020492041233677084, 'subsample': 0.2456645260257502, 'colsample_bytree': 0.6923086437954273}. Best is trial 92 with value: 0.29318860134358105.


----------------------------------------------------------------------------------------------------
Fold: 3
Train Accuracy Score:-0.692542101042502
Valid Accuracy Score:-0.6365
----------Confusion Matrix--------
[[1144  435]
 [ 292  129]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 4
Train Accuracy Score:-0.6922027915931334
Valid Accuracy Score:-0.6135
----------Confusion Matrix--------
[[1112  467]
 [ 306  115]]
----------------------------------------------------------------------------------------------------
Accuracy Score: 0.5999
Custom matrix: 0.25601790908036326
----------------------------------------------------------------------------------------------------
Fold: 0
Train Accuracy Score:-0.5127115369755042
Valid Accuracy Score:-0.214
----------Confusion Matrix--------
[[  12 1568]
 [   4  416]]
-------------------

[I 2024-05-13 14:12:39,252] Trial 99 finished with value: 0.2388793280187512 and parameters: {'booster': 'gblinear', 'lambda': 3.7631678345649176e-05, 'alpha': 0.5700909394908855, 'subsample': 0.5315690751958124, 'colsample_bytree': 0.6372374749087166}. Best is trial 92 with value: 0.29318860134358105.


----------------------------------------------------------------------------------------------------
Fold: 3
Train Accuracy Score:-0.5066559743384121
Valid Accuracy Score:-0.7895
----------Confusion Matrix--------
[[1579    0]
 [ 421    0]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 4
Train Accuracy Score:-0.5093053104444087
Valid Accuracy Score:-0.7165
----------Confusion Matrix--------
[[1363  216]
 [ 351   70]]
----------------------------------------------------------------------------------------------------
Accuracy Score: 0.4729
Custom matrix: 0.2388793280187512


In [ ]:
best_params = study.best_params

CB = make_pipeline(
    XGBClassifier(**best_params)
    )

In [70]:
# Train Catboost Model
_, test_predictions = cross_val_model(train, CB)

----------------------------------------------------------------------------------------------------
Fold: 0
Train Accuracy Score:-0.8185
Valid Accuracy Score:-0.6465
----------Confusion Matrix--------
[[1201  379]
 [ 328   92]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 1
Train Accuracy Score:-0.812875
Valid Accuracy Score:-0.671
----------Confusion Matrix--------
[[1237  343]
 [ 315  105]]
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Fold: 2
Train Accuracy Score:-0.817625
Valid Accuracy Score:-0.659
----------Confusion Matrix--------
[[1208  371]
 [ 311  110]]
----------------------------------------------------------------------------------------------------
--------

# Submission


In [51]:
sample_sub[TARGET] = test_predictions

In [52]:
sample_sub[TARGET].value_counts()

Death
0.2    1786
0.0    1531
0.4    1118
0.6     426
0.8     123
1.0      16
Name: count, dtype: int64

In [47]:
sample_sub[TARGET]  = np.where(sample_sub[TARGET] > 0.7, 1, 0)

In [48]:
sample_sub[TARGET].value_counts()

Death
0    4861
1     139
Name: count, dtype: int64

In [49]:
sample_sub.to_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/SUBMISSION/5_13_4.csv", index=False) #path